In [1]:
import pandas as pd
import os

In [2]:
os.getcwd()

'C:\\Users\\Kenei\\OneDrive\\Documents\\GitHub\\Thinkful\\12.6_Data_Cleaning_and_Validation'

In [3]:
data = pd.read_csv('C:\Users\Kenei\OneDrive\Documents\GitHub\Thinkful\12.6_Data_Cleaning_and_Validation\data.csv')


SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (<ipython-input-3-715b9c4c4934>, line 1)